# Install Packages

In [1]:
!pip install duckdb glob2 polars pyarrow


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Create views

In [2]:
import duckdb as ddb
import os
from glob import glob

# Schritt 1: Verbindung zum Erstellen der Views
con = ddb.connect(database="mimic_v2.db")
count = 0

paths = {
    'ed': 'files/ed',
    'hosp': 'files/hosp',
    'icu': 'files/icu',
}

try:
    for schema, path in paths.items():
        # create schema if not exists
        con.execute(f"CREATE SCHEMA IF NOT EXISTS {schema};")
        csv_files = sorted(glob(os.path.join(path, '*.csv')))
        print(f"Found {len(csv_files)} csv files in {path} for schema {schema}")
        for p in csv_files:
            count += 1
            # derive a safe view name from filename
            fname = os.path.splitext(os.path.basename(p))[0]
            view_name = ''.join(c if c.isalnum() else '_' for c in fname).lower()
            full_view = f"{schema}.{view_name}"
            # create or replace view pointing to the CSV
            sql = f"CREATE OR REPLACE VIEW {full_view} AS SELECT * FROM read_csv_auto('{p}');"
            try:
                con.execute(sql)
                print(f"View-Nr.:{count} Created view {full_view} -> {p}")
            except Exception as e:
                print(f"Failed to create view {full_view} for {p}: {e}")
finally:
    # Sauber schließen
    con.close()
    print('Views erstellt und Verbindung geschlossen.')

# Schritt 2: Read-only Verbindung öffnen
con = ddb.connect(database="mimic_v2.db", read_only=True)
print('Read-only Verbindung geöffnet.')

Found 6 csv files in files/ed for schema ed
View-Nr.:1 Created view ed.diagnosis -> files/ed/diagnosis.csv
View-Nr.:1 Created view ed.diagnosis -> files/ed/diagnosis.csv
View-Nr.:2 Created view ed.edstays -> files/ed/edstays.csv
View-Nr.:3 Created view ed.medrecon -> files/ed/medrecon.csv
View-Nr.:4 Created view ed.pyxis -> files/ed/pyxis.csv
View-Nr.:5 Created view ed.triage -> files/ed/triage.csv
View-Nr.:2 Created view ed.edstays -> files/ed/edstays.csv
View-Nr.:3 Created view ed.medrecon -> files/ed/medrecon.csv
View-Nr.:4 Created view ed.pyxis -> files/ed/pyxis.csv
View-Nr.:5 Created view ed.triage -> files/ed/triage.csv
View-Nr.:6 Created view ed.vitalsign -> files/ed/vitalsign.csv
Found 22 csv files in files/hosp for schema hosp
View-Nr.:6 Created view ed.vitalsign -> files/ed/vitalsign.csv
Found 22 csv files in files/hosp for schema hosp
View-Nr.:7 Created view hosp.admissions -> files/hosp/admissions.csv
View-Nr.:8 Created view hosp.d_hcpcs -> files/hosp/d_hcpcs.csv
View-Nr.:9

# Helper Functions

In [3]:
import polars as pl
import pandas as pd

# sql helper function
def sql(q: str):
    """Run a sql query on the open DuckDB connection and return a Polars DataFrame."""
    pl.Config.set_tbl_rows(-1)
    pl.Config.set_tbl_cols(-1)
    try:
        return con.execute(q).pl()
    except Exception as e:
        print(f"SQL query failed: {e}")
        return pl.DataFrame()
    
def show(df: pl.DataFrame, limit: bool = False):
    """Display a Polars DataFrame with configurable column width and row limit."""
    if limit:
        pd.set_option("display.max_colwidth", 50)
        pd.set_option("display.max_rows", 20)
    else:
        pd.set_option("display.max_colwidth", None)
        pd.set_option("display.max_rows", None)
    display(df.to_pandas())

# Overview & Metadata

In [4]:
# SQL
df = sql("SHOW ALL TABLES")
show(df)

,database,schema,name,column_names,column_types,temporary
0,mimic_v2,ed,diagnosis,"[subject_id, stay_id, seq_num, icd_code, icd_version, icd_title]","[BIGINT, BIGINT, BIGINT, VARCHAR, BIGINT, VARCHAR]",False
1,mimic_v2,ed,edstays,"[subject_id, hadm_id, stay_id, intime, outtime, gender, race, arrival_transport, disposition]","[BIGINT, BIGINT, BIGINT, TIMESTAMP, TIMESTAMP, VARCHAR, VARCHAR, VARCHAR, VARCHAR]",False
2,mimic_v2,ed,medrecon,"[subject_id, stay_id, charttime, name, gsn, ndc, etc_rn, etccode, etcdescription]","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR, VARCHAR, BIGINT, VARCHAR, VARCHAR]",False
3,mimic_v2,ed,pyxis,"[subject_id, stay_id, charttime, med_rn, name, gsn_rn, gsn]","[BIGINT, BIGINT, TIMESTAMP, BIGINT, VARCHAR, BIGINT, VARCHAR]",False
4,mimic_v2,ed,triage,"[subject_id, stay_id, temperature, heartrate, resprate, o2sat, sbp, dbp, pain, acuity, chiefcomplaint]","[BIGINT, BIGINT, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, VARCHAR, DOUBLE, VARCHAR]",False
5,mimic_v2,ed,vitalsign,"[subject_id, stay_id, charttime, temperature, heartrate, resprate, o2sat, sbp, dbp, rhythm, pain]","[BIGINT, BIGINT, TIMESTAMP, DOUBLE, DOUBLE, DOUBLE, DOUBLE, BIGINT, BIGINT, VARCHAR, VARCHAR]",False
6,mimic_v2,hosp,admissions,"[subject_id, hadm_id, admittime, dischtime, deathtime, admission_type, admit_provider_id, admission_location, discharge_location, insurance, language, marital_status, race, edregtime, edouttime, hospital_expire_flag]","[BIGINT, BIGINT, TIMESTAMP, TIMESTAMP, TIMESTAMP, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, TIMESTAMP, TIMESTAMP, BIGINT]",False
7,mimic_v2,hosp,d_hcpcs,"[code, category, long_description, short_description]","[VARCHAR, BIGINT, VARCHAR, VARCHAR]",False
8,mimic_v2,hosp,d_icd_diagnoses,"[icd_code, icd_version, long_title]","[VARCHAR, BIGINT, VARCHAR]",False
9,mimic_v2,hosp,d_icd_procedures,"[icd_code, icd_version, long_title]","[VARCHAR, BIGINT, VARCHAR]",False


# SQL Explore Data

## Patient Data

In [5]:
# first 5 rows of patients table
df = sql("SELECT * from hosp.patients")
show(df, limit=True)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaT
2,10000058,F,33,2168,2020 - 2022,NaT
3,10000068,F,19,2160,2008 - 2010,NaT
4,10000084,M,72,2160,2017 - 2019,2161-02-13
...,...,...,...,...,...,...
364622,19999828,F,46,2147,2017 - 2019,NaT
364623,19999829,F,28,2186,2008 - 2010,NaT
364624,19999840,M,58,2164,2008 - 2010,2164-09-17
364625,19999914,F,49,2158,2017 - 2019,NaT


In [6]:
# Anzahl Patienten, Aufenthalte
sql("""
SELECT COUNT(DISTINCT subject_id) AS patients,
       COUNT(DISTINCT hadm_id) AS admissions
FROM hosp.admissions;

""")

patients,admissions
i64,i64
223452,546028


## Diagnosis Data

In [7]:
# first 5 rows of diagnosis table
df = sql("""SELECT *
         from ed.diagnosis order by icd_code
         """)
show(df,limit=True)

,subject_id,stay_id,seq_num,icd_code,icd_version,icd_title
0,10427677,35873964,1,0020,9,TYPHOID FEVER
1,11545281,36308244,1,0030,9,SALMONELLA ENTERITIS
2,19906243,39205868,1,0059,9,FOOD POISONING NOS
3,19732857,35260270,1,0059,9,FOOD POISONING NOS
4,18106634,34272117,1,0059,9,FOOD POISONING NOS
...,...,...,...,...,...,...
899045,11548266,37524655,3,Z9981,10,Dependence on supplemental oxygen
899046,12070608,32123100,3,Z9981,10,Dependence on supplemental oxygen
899047,17396346,35428975,2,Z9981,10,Dependence on supplemental oxygen
899048,16865766,39674308,4,Z9981,10,Dependence on supplemental oxygen


In [8]:
# count of distinct icd titles (diagnoses)
df = sql("""SELECT count(icd_title), icd_title 
         from ed.diagnosis
         group by 2
         order by 1 desc""")
show(df, limit=True)

,count(icd_title),icd_title
0,26816,HYPERTENSION NOS
1,21264,Essential (primary) hypertension
2,13016,"Chest pain, unspecified"
3,12398,CHEST PAIN NOS
4,12026,DIABETES UNCOMPL ADULT
...,...,...
13167,1,"Streptococcus, group A, causing diseases class..."
13168,1,"Displ seg fx shaft of l tibia, init for opn fx..."
13169,1,Prsn brd/alit pk-up/van injured in nonclsn trn...
13170,1,"Disp fx of body of scapula, right shoulder, in..."


In [9]:
sql("""SELECT *
         from hosp.d_icd_diagnoses
         -- group by 2
         -- order by 1 desc
         where icd_code like '%0'
         """)
show(df, limit=True)

,count(icd_title),icd_title
0,26816,HYPERTENSION NOS
1,21264,Essential (primary) hypertension
2,13016,"Chest pain, unspecified"
3,12398,CHEST PAIN NOS
4,12026,DIABETES UNCOMPL ADULT
...,...,...
13167,1,"Streptococcus, group A, causing diseases class..."
13168,1,"Displ seg fx shaft of l tibia, init for opn fx..."
13169,1,Prsn brd/alit pk-up/van injured in nonclsn trn...
13170,1,"Disp fx of body of scapula, right shoulder, in..."


In [10]:
# count of distinct icd titles (diagnoses)
df = sql("""SELECT *
         from hosp.diagnoses_icd""")
show(df,limit=True)

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
6364483,19999987,23865745,7,41401,9
6364484,19999987,23865745,8,78039,9
6364485,19999987,23865745,9,0413,9
6364486,19999987,23865745,10,36846,9


In [11]:
# avg length of stay in days
sql("""
SELECT ROUND(AVG(EXTRACT(EPOCH FROM (dischtime - admittime))/3600/24), 2) AS avg_los_days
FROM hosp.admissions;
""")

avg_los_days
f64
4.76


In [12]:
# diagnoses per admission
sql("""
SELECT COUNT(*) / COUNT(DISTINCT hadm_id) AS avg_diagnoses_per_admission
FROM hosp.diagnoses_icd;
""")

avg_diagnoses_per_admission
f64
11.66732


# Vizusalization
Patient Data

In [13]:
! pip install plotly nbformat


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Patients

In [14]:
age_range = sql("""SELECT max(anchor_age) as max_age,
                     min(anchor_age) as min_age,
                     AVG(anchor_age) as avg_age
                from hosp.patients;""")
show(age_range)

,max_age,min_age,avg_age
0,91,18,48.875097


In [15]:
import plotly.express as px
# load patient data
df_patients = sql("SELECT * from hosp.patients;")

# Beispiel: df hat Spalten ["patient_id", "age"]
fig = px.histogram(df_patients, x="anchor_age", nbins=70, title="Age of Patients Histogram")
fig.update_layout(bargap=0.2, xaxis_title="Age", yaxis_title="Count of Patients")
fig.show()


In [16]:
# patients with admissions and demographics
df = sql("""
    SELECT 
        p.subject_id,
        p.gender,
        p.anchor_age,
        a.race,
        a.admission_type,
        a.admission_location,
        a.insurance,
        a.marital_status,
        a.language
    FROM hosp.patients p
    JOIN hosp.admissions a ON p.subject_id = a.subject_id
""")
show(df,limit=True)

,subject_id,gender,anchor_age,race,admission_type,admission_location,insurance,marital_status,language
0,10876550,M,79,WHITE,EW EMER.,EMERGENCY ROOM,Medicare,MARRIED,English
1,10876573,M,32,WHITE,EW EMER.,EMERGENCY ROOM,Medicaid,SINGLE,English
2,10876693,M,62,BLACK/AFRICAN AMERICAN,EW EMER.,EMERGENCY ROOM,Private,MARRIED,English
3,10876693,M,62,BLACK/AFRICAN AMERICAN,EU OBSERVATION,EMERGENCY ROOM,Medicare,MARRIED,English
4,10876750,F,73,UNKNOWN,URGENT,TRANSFER FROM HOSPITAL,Medicare,MARRIED,English
...,...,...,...,...,...,...,...,...,...
546023,19341569,M,50,BLACK/AFRICAN AMERICAN,EU OBSERVATION,WALK-IN/SELF REFERRAL,Medicare,SINGLE,English
546024,19341569,M,50,BLACK/AFRICAN AMERICAN,EU OBSERVATION,WALK-IN/SELF REFERRAL,Medicare,SINGLE,English
546025,19341569,M,50,BLACK/AFRICAN AMERICAN,EU OBSERVATION,WALK-IN/SELF REFERRAL,Medicare,SINGLE,English
546026,19341569,M,50,BLACK/AFRICAN AMERICAN,EU OBSERVATION,WALK-IN/SELF REFERRAL,Medicare,SINGLE,English


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# =============================================================================
# 2. ALTERS- UND GESCHLECHTSVERTEILUNG (POPULATION PYRAMID)
# =============================================================================
age_bins = [0, 18, 30, 40, 50, 60, 70, 80, 90, 100, 120]
df_pyramid = df.with_columns(
    pl.col("anchor_age").cut(age_bins).alias("age_group")
)

pyramid_data = (df_pyramid
    .group_by(["age_group", "gender"])
    .agg(pl.count().alias("count"))
    .sort("age_group")
)

# Erstelle Population Pyramid
male_data = pyramid_data.filter(pl.col("gender") == "M")
female_data = pyramid_data.filter(pl.col("gender") == "F")

fig1 = go.Figure()
fig1.add_trace(go.Bar(
    y=male_data["age_group"].cast(str).to_list(),
    x=[-x for x in male_data["count"].to_list()],  # Konvertiere zu Liste für Negation
    name='Male',
    orientation='h',
    marker=dict(color='steelblue')
))
fig1.add_trace(go.Bar(
    y=female_data["age_group"].cast(str).to_list(),
    x=female_data["count"].to_list(),
    name='Female',
    orientation='h',
    marker=dict(color='salmon')
))

fig1.update_layout(
    title='Population Pyramid: Age and Gender Distribution',
    barmode='relative',
    bargap=0.1,
    xaxis_title='Number of Patients',
    yaxis_title='Age Group',
    height=600
)
fig1.show()

/var/folders/z6/l14kg6kd267dqw00p40_86jc0000gn/T/ipykernel_42182/2739793193.py:13: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)



# Diagnosis

In [18]:
# sql query to get diagnosis data
df = sql("""SELECT count(icd_title) as count, icd_title 
         from ed.diagnosis
         group by 2
         order by 1 desc limit 20;""")

# Top 20 most frequent diagnoses

fig1 = px.bar(df, x="count", y="icd_title", 
              title="Top 20 Most Frequent Diagnoses",
              orientation='h')
fig1.update_layout(yaxis={'categoryorder':'total ascending'}, 
                   height=600, yaxis_title="", xaxis_title="Number of Cases")
fig1.show()